# Femnist Table 1

TJ Kim

1.17.22

#### Summary:
- Make a table for Benign transferability and inter-boundary distance for following models
- Local benign, fedavg benign, fedEM benign

In [2]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [3]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Local Benign

In [283]:
setting = 'local'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "femnist"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=300)

==> Clients initialization..
===> Building data iterators..


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 719/719 [00:01<00:00, 666.77it/s]


===> Initializing clients..


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 719/719 [00:06<00:00, 111.41it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 4.147 | Train Acc: 1.769% |Test Loss: 4.146 | Test Acc: 1.699% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


Generate Dataset to be used throughout all analysis.

In [284]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 

In [285]:
dataloader = Custom_Dataloader(data_x, data_y)

Load local model.

In [295]:
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
    args_.save_path ='weights/final/femnist/fig1_take3/local_adv/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
    weights = np.load('weights/final/femnist/fig1_take3/local_adv/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
    args_.save_path = 'weights/final/femnist/fig1_take3/FedAvg_adv/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load('weights/final/femnist/fig1_take3/FedAvg_adv/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
    args_.save_path = 'weights/final/femnist/figperturb/fedem_avg_p0_1/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedem_benign/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
    weights = np.load("weights/final/femnist/figperturb/fedem_avg_p0_1/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [296]:
for i in range(weights.shape[0]):
    print(i, weights[i])

0 [1.00]
1 [1.00]
2 [1.00]
3 [1.00]
4 [1.00]
5 [1.00]
6 [1.00]
7 [1.00]
8 [1.00]
9 [1.00]
10 [1.00]
11 [1.00]
12 [1.00]
13 [1.00]
14 [1.00]
15 [1.00]
16 [1.00]
17 [1.00]
18 [1.00]
19 [1.00]
20 [1.00]
21 [1.00]
22 [1.00]
23 [1.00]
24 [1.00]
25 [1.00]
26 [1.00]
27 [1.00]
28 [1.00]
29 [1.00]
30 [1.00]
31 [1.00]
32 [1.00]
33 [1.00]
34 [1.00]
35 [1.00]
36 [1.00]
37 [1.00]
38 [1.00]
39 [1.00]
40 [1.00]
41 [1.00]
42 [1.00]
43 [1.00]
44 [1.00]
45 [1.00]
46 [1.00]
47 [1.00]
48 [1.00]
49 [1.00]
50 [1.00]
51 [1.00]
52 [1.00]
53 [1.00]
54 [1.00]
55 [1.00]
56 [1.00]
57 [1.00]
58 [1.00]
59 [1.00]
60 [1.00]
61 [1.00]
62 [1.00]
63 [1.00]
64 [1.00]
65 [1.00]
66 [1.00]
67 [1.00]
68 [1.00]
69 [1.00]
70 [1.00]
71 [1.00]
72 [1.00]
73 [1.00]
74 [1.00]
75 [1.00]
76 [1.00]
77 [1.00]
78 [1.00]
79 [1.00]
80 [1.00]
81 [1.00]
82 [1.00]
83 [1.00]
84 [1.00]
85 [1.00]
86 [1.00]
87 [1.00]
88 [1.00]
89 [1.00]
90 [1.00]
91 [1.00]
92 [1.00]
93 [1.00]
94 [1.00]
95 [1.00]
96 [1.00]
97 [1.00]
98 [1.00]
99 [1.00]
100 [1.00]

In [297]:
# desired_servers = [3,64,126,162,245,38,61,81,90,119]
desired_servers = [265, 24,39,64,109]
new_models_test = []

for i in range(len(desired_servers)):
    new_models_test += [models_test[i]]

new_models_test = models_test

Set Up Transfer Attack Scenario

In [298]:
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]


In [304]:
# Run Measurements for both targetted and untargeted analysis
new_num_models = len(new_models_test)
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 4.5


for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    
    dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    
    t1 = Transferer(models_list=new_models_test, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    
    # Perform Attacks
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    
    
    
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Miss attack
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 10
	 Adv idx: 11
	 Adv idx: 12
	 Adv idx: 13
	 Adv idx: 14
	 Adv idx: 15
	 Adv idx: 16
	 Adv idx: 17
	 Adv idx: 18
	 Adv idx: 19
	 Adv idx: 20
	 Adv idx: 21
	 Adv idx: 22
	 Adv idx: 23
	 Adv idx: 24
	 Adv idx: 25
	 Adv idx: 26
	 Adv idx: 27
	 Adv idx: 28
	 Adv idx: 29
	 Adv idx: 30
	 Adv idx: 31
	 Adv idx: 32
	 Adv idx: 33
	 Adv idx: 34
	 Adv idx: 35
	 Adv idx: 36
	 Adv idx: 37
	 Adv idx: 38
	 Adv idx: 39


Print Relevant Information

In [305]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [306]:
print('adv_target:', avg_nondiag(adv_target))
print('adv_miss:', avg_nondiag(adv_miss))
print('orig_acc:', np.mean(np.diagonal(orig_acc)))

adv_target: 0.021070879860781135
adv_miss: 0.672129987180233
orig_acc: 0.7393902689218521


In [307]:
print(np.mean(orig_acc[:20,:20],axis=1))

[0.79 0.86 0.71 0.55 0.66 0.84 0.67 0.53 0.88 0.82 0.73 0.86 0.84 0.79
 0.68 0.96 0.75 0.84 0.88 0.69]


In [308]:
print(np.mean(adv_miss[:20,:20],axis=1))

[0.79 0.78 0.62 0.46 0.56 0.74 0.53 0.42 0.88 0.74 0.67 0.80 0.73 0.69
 0.62 0.96 0.69 0.80 0.78 0.66]


Set Up Interboundary Measure Scenario

In [276]:
clients[6].test_iterator.dataset.targets

tensor([23,  4, 25,  3,  7, 17, 28, 28, 31,  7, 30, 24, 22, 27, 35,  5, 28, 28,
         7,  4, 30,  8,  4,  2, 14,  9, 60, 46,  7, 11,  1, 24, 38, 24, 22,  0,
        30, 12,  6,  0,  9, 19,  7, 49, 59, 43,  3, 24, 28])

In [277]:
clients[1].test_iterator.dataset.targets

tensor([ 7, 16,  7,  5,  9,  5,  2,  8, 19,  1,  6, 10,  8, 50,  8,  9, 34,  6,
         9,  8,  7,  1,  7,  6, 17, 26,  5,  7,  3, 49, 12, 27,  3, 37, 23,  9,
         1])

#### Logs

In [ ]:
# Global Validation Dataset

# Local - Benign
local_adv_miss = 0.21# temp
local_adv_target = 0.03#temp
local_orig_acc = 0.61
local_ibdist_legit = 7.615
local_ibdist_adv = 41.505625

# Fedavg - Benign
fedavg_adv_miss = 0.34 # temp
fedavg_adv_target = 0.10 # temp
fedavg_orig_acc = 0.72
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0 # 15.55

# FedEM - Benign
fedem_adv_miss = 0.33 # temp
fedem_adv_target = 0.10 # temp
fedem_orig_acc = 0.67
fedem_ibdist_legit = 0.01125
fedem_ibdist_adv = 13.613125

# Local - adv
local_adv_miss = 
local_adv_target = 
local_orig_acc = 
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 
fedavg_adv_target =  
fedavg_orig_acc = 
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 
fedem_adv_target = 
fedem_orig_acc = 
fedem_ibdist_legit = 
fedem_ibdist_adv = 

In [ ]:
# Local Validation Dataset

# Local
local_adv_miss = 0.37
local_adv_target = 0.06
local_orig_acc = 0.99
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - Benign
fedavg_adv_miss = 0.00
fedavg_adv_target = 0.85
fedavg_orig_acc = 0.94
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.10
fedem_adv_target = 0.44
fedem_orig_acc = 0.94
fedem_ibdist_legit = 
fedem_ibdist_adv = 

# Local - adv
local_adv_miss = 0.29
local_adv_target = 0.05
local_orig_acc = 0.77
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 0.33
fedavg_adv_target =  0.24
fedavg_orig_acc = 0.87
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.49
fedem_adv_target = 0.07
fedem_orig_acc = 0.84
fedem_ibdist_legit = 
fedem_ibdist_adv = 

In [ ]:
# Local Test (Train) Dataset

# Local
local_adv_miss = 0.38
local_adv_target = 0.06
local_orig_acc = 0.52
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - Benign
fedavg_adv_miss = 0.00
fedavg_adv_target = 0.85
fedavg_orig_acc = 0.81
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.10
fedem_adv_target = 0.46
fedem_orig_acc = 0.84
fedem_ibdist_legit = 
fedem_ibdist_adv = 

# Local - adv
local_adv_miss = 0.30
local_adv_target = 0.05
local_orig_acc = 0.46
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 0.26
fedavg_adv_target = 0.30
fedavg_orig_acc = 0.74
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.42
fedem_adv_target = 0.09
fedem_orig_acc = 0.74
fedem_ibdist_legit = 
fedem_ibdist_adv = 